In [250]:
###%reset -f
import pandas as pd
import numpy as np 
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from lightgbm import LGBMClassifier
import tensorflow as tf 
from sklearn.metrics import roc_curve, auc,roc_auc_score


In [455]:
ip=pd.read_csv('D:/Prasanna/Prasanna/sepmodels/attr.csv',sep=",") 
oot=pd.read_csv('D:/Prasanna/Prasanna/sepmodels/oot_attr.csv',sep=",")  
 

In [502]:
####cat list
which_cat = ip.select_dtypes(include = [np.object]).columns.values 
## strip out dates, id's,acct numbers etc which are not deemed  categorical variables 
cat_list1 = which_cat.tolist() ## convert into a list
cat_list1=[column for column in cat_list if column not in all_null_list]


In [505]:
def cat_data_stats(data,cols):
    column=data[cols]
    cat_dict=dict(column.value_counts())
    cat_missing_values=sum(pd.isnull(column))
    cat_missing_perc=cat_missing_values/ip.shape[0]
    return(cat_dict,cat_missing_values,cat_missing_perc)
 

## Defines list to hold respective values
   
header=[]
var_list=[]
freq_list=[]
percentage=[]
mode_cat=[]
cum_freq=[]
cat_miss=[]
cat_miss_percentage=[]
cat_miss1=[]
cat_miss_percentage1=[]


## Iterate thru all column values to get the respective column statsistics
for column_name in cat_list1:
    #print("Column_name:",column_name)
    cat_dict,cat_missing_values,cat_missing_perc=cat_data_stats(ip,column_name)
    cat_miss1.append(cat_missing_values)
    cat_miss_percentage1.append(cat_missing_perc)
    ### find the mode ..ie maximum number of occurances of the factors in 
    ### the categorical variable
    mode_var=max(cat_dict, key=cat_dict.get)
       
    for Var_category in cat_dict.keys():
        cat_miss.append(cat_missing_values)
        cat_miss_percentage.append(cat_missing_perc)
        header.append(column_name)               ## Adding  variable names 
        var_list.append(Var_category)            ## Appending values ie the key from dict

    for frequency in cat_dict.values():  
        freq_list.append(frequency)              ## Appending frequency
        percentage.append(frequency/sum(cat_dict.values())) ##
        mode_cat.append(mode_var)  ## Mode appended
        
        
        
    
##Creating  a PD to hold the cat column stats
   
#### 1) Create dictionary to hold respective stats      
basic_stats_cat_rpt = {   'Variable'  : header,
                      'Category'  : var_list,
                      'Frequency' : freq_list,
                      'Percentage': percentage,
                      'Missing_Values':cat_miss,
                      'Missing_values %':cat_miss_percentage,
                      'Mode'      : mode_cat
                      }

###  2) Create a pandas dataframe using above dictionary

basic_stats_cat = pd.DataFrame(basic_stats_cat_rpt, columns = 
['Variable','Category','Frequency','Percentage','Missing_Values','Missing_values %','Mode'])

##Write as csv to a folder

basic_stats_cat.to_csv('D:/Prasanna/Prasanna/sepmodels/basic_stats_cat.csv', encoding='utf-8')


In [445]:
which_num = ip.select_dtypes(include = [np.float64, np.int64]).columns.values
num_list1 = which_num.tolist()


In [449]:
def hampel_lim (data,cols):
    column=data[cols].tolist()
    median=np.nanmedian(column)
    MAD=np.nanmedian([abs(x - median) for x in column])
    HUL=np.nanmedian(column)+3*(np.nanmedian([abs(x - median) for x in column]))
    HLL=np.nanmedian(column)-3*(np.nanmedian([abs(x - median) for x in column]))
    return(median,MAD,HUL,HLL)
    
    
############  mu_3siglim function#########################################
##input   :  dataframe, columns
##output  :  Upper and lower limits    
    
def mu_3siglim(data,cols):
    column=data[cols].tolist()
    sd=np.nanstd(column,axis=0)
    UL=np.nanmean(column)+3*sd # Upper limit
    LL=np.nanmean(column)-3*sd # Lower limit
    return(UL,LL)
 
############  percentile function#########################################
##input    :  dataframe, columns
##output   : percentiles     
 
def percentiles(data,cols):
    column=data[cols]
    return(np.nanpercentile(column, [1, 2, 5, 10, 20, 30, 40 , 50,
                         60, 70, 75,80, 90, 95, 96, 97, 98,99]))

 
########### Basic statistics function ###################################
## input  : dataframe,columns
## output : Number of records,minimum,maximum,ranges,sum,variance & standard deviation
 
def basic_statistics(data,cols):
    column=data[cols].tolist()
    N=len(column)
    minimum=np.nanmin(column)
    maximum=np.nanmax(column)
    ranges=maximum-minimum
    sums=np.nansum(column)
    mean=np.nanmean(column)
    var=np.nanvar(column)
    stddev=np.nanstd(column)
    return(N,minimum,maximum,ranges,sums,mean,var,stddev)
    
########## Missing values function ########################################
## input  : dataframe ,columns
## output : Missing values  

def missing_values(data,cols):
    column=data[cols]
    missing_value= len(column.index)-column.count()
    return(missing_value)
    
########## Zero values function ########################################
## input  : dataframe ,columns
## output : Zero values  
    
    
def zero_values(data,cols):
    column=data[cols]
    zero_val=(column == 0).astype(int).sum(axis=0)
    return(zero_val)
        


###################################End###########################################

#########Define lists to store basic stats.The lists have been named accordingly 
#########to store respective stats 


header=[]
median_row=[]
mad_row=[]
HUL_row=[]
HLL_row=[]
UL_row=[]
LL_row=[]    
missing_row=[]
zeros_row=[]
N_row=[]
min_row=[]
max_row=[]
percent_row=[]
range_row=[]
sum_row=[]
mean_row=[]
var_row=[]
stddev_row=[]
percent_1row=[]
percent_2row=[]
percent_5row=[]
percent_10row=[]
percent_20row=[]
percent_30row=[]
percent_40row=[]
percent_50row=[]
percent_60row=[]
percent_70row=[]
percent_75row=[]
percent_80row=[]
percent_90row=[]
percent_95row=[]
percent_96row=[]
percent_97row=[]
percent_98row=[]
percent_99row=[]

nochange=[]  
binary_list=[]                   
missing_percentage_row=[] 
sd_zero=[]

se_row=[]
CI_row=[]
coef_var=[]

kurtosis_row=[]
skewness_row=[]


In [454]:
from scipy import stats 
from scipy import nanmean
import scipy
from scipy.stats.stats import pearsonr

df=ip
for column_name in num_list1:
    median,MAD,HUL,HLL=hampel_lim(df,column_name)
    UL,LL= mu_3siglim(df,column_name)
    percent=percentiles(df,column_name)
    missing_value=missing_values(df,column_name)
    zero_value=zero_values(df,column_name)
    N,minimum,maximum,ranges,sums,mean,var,stddev=basic_statistics(df,column_name)
    
    ##  if maximum  = minimum then no change 
    ## then store value by appending    
    if minimum==maximum:
        nochange.append(column_name)
    
    ##  If sum  of count of zeros amd one are equal to count of number
    ##  of rows then data is only binary  ( ie 0 and 1 only)
    ##  then store value by appending        
    if df[column_name].tolist().count(0) \
    +  df[column_name].tolist().count(1)==df.shape[0]:
        binary_list.append(column_name)
        
    ## If standard deviation = 0 then store the value by appending         
    if stddev==0:
        sd_zero.append(column_name)
    
    ## Missing % = count of missing values/total number of rows
    
    missing_percentage=missing_value/(df.shape[0])  
    header.append(column_name)
    median_row.append(median)
    mad_row.append(MAD)
    HUL_row.append(HUL)
    HLL_row.append(HLL)
    
    UL_row.append(UL)
    LL_row.append(LL)
    
    missing_row.append(missing_value)
    missing_percentage_row.append(missing_percentage)
    zeros_row.append(zero_value)
    N_row.append(N)
    min_row.append(minimum)
    max_row.append(maximum)
    percent_row.append(percent)
    range_row.append(ranges)
    sum_row.append(sums)
    mean_row.append(mean)
    
    ############SE,CI and coef_var #### needs to be imputed 
    se_row.append("None")
    CI_row.append("None")
    coef_var.append("None")
    ######################################################    
    
    var_row.append(var)
    stddev_row.append(stddev)
    percent_1row.append(percent[0])
    percent_2row.append(percent[1])
    percent_5row.append(percent[2])
    percent_10row.append(percent[3])
    percent_20row.append(percent[4])
    percent_30row.append(percent[5])
    percent_40row.append(percent[6])
    percent_50row.append(percent[7])
    percent_60row.append(percent[8])
    percent_70row.append(percent[9])
    percent_75row.append(percent[10])
    percent_80row.append(percent[11])
    percent_90row.append(percent[12])
    percent_95row.append(percent[13])
    percent_96row.append(percent[14])
    percent_97row.append(percent[15])
    percent_98row.append(percent[16])
    percent_99row.append(percent[17])
    df_no_missing=df[column_name].dropna()
    kurtosis_row.append(scipy.stats.kurtosis(df_no_missing,fisher=False))
    skewness_row.append(scipy.stats.skew(df_no_missing))
    
        
#######Creating   PD dataframe containing basic stats############        
### 1)  Each variable is stored as a dictionary ie example : key   : Mean
###                                                          value : mean_row        
        
basic_stats_report = {'Variable': header,'Number of rows':N_row, 'Zeros'    : zeros_row,  
                      'Missing' :missing_row,'Minimum'   :min_row,'Maximum' : max_row,
                      'Range'   :range_row  ,'Sum'       :sum_row,'Median'  : median_row,
                      'Mean'    :mean_row   ,'SE.mean'   :se_row ,'CI.mean.0.95':CI_row, 
                      'Variance':var_row    ,'stddev'    :stddev_row,'coef.var':coef_var,
                      '1_Percentile'  : percent_1row,    '2_Percentile'  : percent_2row,
                      '5_Percentile'  : percent_5row,    '10_Percentile' : percent_10row,
                      '20_Percentile' : percent_20row,   '30_Percentile' : percent_30row,
                      '40_Percentile' : percent_40row,   '50_Percentile' : percent_50row,
                      '60_Percentile' : percent_60row,   '70_Percentile' : percent_70row,
                      '75_Percentile' : percent_75row,   '80_Percentile' : percent_80row,
                      '90_Percentile' : percent_90row,   '95_Percentile' : percent_95row,
                      '96_Percentile' : percent_96row,   '97_Percentile' : percent_97row,
                      '98_Percentile' : percent_98row,   '99_Percentile' : percent_99row,
                      'UL'            : UL_row,'LL'      : LL_row,'MAD':mad_row,
                      'HUL'           : HUL_row,'HLL'    : HLL_row
                      
                      }
  
 
 
basic_stats_num = pd.DataFrame(basic_stats_report,columns =['Variable','Number of rows','Zeros',  
                                'Missing','Minimum','Maximum','Range','Sum','Median',
                                'Mean','SE.mean','CI.mean.0.95','Variance'  ,'stddev','coef.var',
                                '1_Percentile','2_Percentile','5_Percentile','10_Percentile',
                                '20_Percentile','30_Percentile','40_Percentile','50_Percentile',
                                '60_Percentile','70_Percentile','75_Percentile','80_Percentile',
                                '90_Percentile','95_Percentile','96_Percentile','97_Percentile',
                                '98_Percentile','99_Percentile','UL','LL','MAD','HUL','HLL'
                                ]
                                  )
                      

                     
basic_stats_num.to_csv('D:/Prasanna/Prasanna/sepmodels/basic_stats_num.csv', encoding='utf-8')  

C:\Users\zcoe590\AppData\Roaming\Python\Python36\site-packages\numpy\lib\function_base.py:3250: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)
C:\Users\zcoe590\AppData\Roaming\Python\Python36\site-packages\numpy\lib\nanfunctions.py:1545: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
C:\Users\zcoe590\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: Mean of empty slice
C:\Users\zcoe590\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: Mean of empty slice
C:\Users\zcoe590\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: All-NaN axis encountered
C:\Users\zcoe590\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: All-NaN axis encountered
C:\Users\zcoe590\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
C:\Use

In [372]:
# Total number of columns
columns=ip.columns.tolist()
columns_dict={'columns': columns}
columns_pd  = pd.DataFrame(columns_dict,columns = (['columns']))
columns_pd.to_csv('D:/Prasanna/Prasanna/sepmodels/allvariables.csv', encoding='utf-8') 

In [379]:
columns=q_list
columns_dict={'columns': columns}
columns_pd  = pd.DataFrame(columns_dict,columns = (['columns']))
columns_pd.to_csv('D:/Prasanna/Prasanna/sepmodels/qlist.csv', encoding='utf-8') 

columns=avg_list
columns_dict={'columns': columns}
columns_pd  = pd.DataFrame(columns_dict,columns = (['columns']))
columns_pd.to_csv('D:/Prasanna/Prasanna/sepmodels/avglist.csv', encoding='utf-8') 

In [485]:
## Get numerical variables 
which_num = ip[var_list].select_dtypes(include = [np.float64, np.int64]).columns.values
num_list = which_num.tolist() ## convert into a list
##remove all null
all_null_list=[col for col in ip[var_list].columns if ip[col].isnull().all()]  
num_list=[column for column in num_list if column not in all_null_list]

####cat list
which_cat = ip[var_list].select_dtypes(include = [np.object]).columns.values 
## strip out dates, id's,acct numbers etc which are not deemed  categorical variables 
cat_list = which_cat.tolist() ## convert into a list
cat_list=[column for column in cat_list if column not in all_null_list]

print(len(num_list))
print(len(cat_list))
print(len(all_null_list))



852
238
98


In [487]:
###############################################################################
X_train, X_test, y_train, y_test = train_test_split(ip, ip['attrition_flag'], test_size=0.35, random_state=2018, stratify=ip[' attrition_flag'])
 
#################count columns#################################################
count_cols = [col for col in num_list if 'cnt' in col]
print("count list " , len(count_cols))

count_list=[cols for cols in count_cols if X_train[cols].isnull().sum()/X_train.shape[0] <= 0.9]  
print("count  after dropping nulls greater than 80 %" , len(count_list))
###################flag columns################################################
flag_cols = [col for col in num_list if 'flag' in col]
print("flag_cols " , len(flag_cols))
flag_list=[cols for cols in flag_cols if X_train[cols].isnull().sum()/X_train.shape[0] <= 0.9]  
print("count  after dropping nulls greater than 80 %" , len(flag_list))

####################new num list###############################################

print("Num_list is :" , len(num_list))
num_list=[cols for  cols in num_list if cols not in count_list]
num_list=[cols for  cols in num_list if cols not in flag_list]
print("Num_list is :" , len(num_list))

################### Num list null values ######################################

num_list=[cols for cols in num_list if X_train[cols].isnull().sum()/X_train.shape[0] <= 0.9]  
print("count  after dropping nulls greater than 80 %" , len(num_list))


count list  387
count  after dropping nulls greater than 80 % 281
flag_cols  14
count  after dropping nulls greater than 80 % 14
Num_list is : 852
Num_list is : 557
count  after dropping nulls greater than 80 % 318


In [488]:
columns=num_list+count_list+
columns_dict={'columns': columns}
columns_pd  = pd.DataFrame(columns_dict,columns = (['columns']))
columns_pd.to_csv('D:/Prasanna/Prasanna/sepmodels/numlist_nulls.9.csv', encoding='utf-8') 

 

In [489]:
################### replace null values #######################################
 
for cols in median_list:
    #print(X_train[cols].head(10))
    X_train[cols].fillna(np.mean(X_train[cols]),inplace=True)
    X_test[cols].fillna(np.mean(X_test[cols]),inplace=True)
    oot[cols].fillna(np.mean(oot[cols]),inplace=True)

nanCounter = np.isnan(X_train[median_list]).sum()    
print(nanCounter)    

 

for cols in num_list:
    #print(X_train[cols].head(10))
    X_train[cols].fillna(0,inplace=True)
    X_test[cols].fillna(0,inplace=True)
    oot[cols].fillna(0,inplace=True)
for cols in count_list:
    #print(X_train[cols].head(10))
    X_train[cols].fillna(0,inplace=True)
    X_test[cols].fillna(0,inplace=True)
    oot[cols].fillna(0,inplace=True)
for cols in flag_list:
    #print(X_train[cols].head(10))
    X_train[cols].fillna(0,inplace=True)
    X_test[cols].fillna(0,inplace=True)
    oot[cols].fillna(0,inplace=True)
    
################## check for null values ######################################    
#nanCounter = np.isnan(X_train[num_list+count_list+flag_list]).sum()    
#print(nanCounter)    


C:\Users\zcoe590\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


cust_age    0
tenure      0
dtype: int64


In [490]:
 

################### SD check   ################################################ 
print(" Total number of numerical features before checking for SD=0 " ,len(num_list))
num_list=[column for column in num_list  if np.nanstd(X_train[column],axis=0) !=0.0 ]
print("Total number of numerical features after checking for SD=0 "  , len(num_list))

num_list=[column for column in num_list  if max(X_train[column])!=min(X_train[column])]
print("Total number of numerical features after checking for max=min "  , len(num_list))

print(" Before unqiue value treatment :",len(num_list))
num_list=[column for column in num_list  if len(X_train[column].unique()) >= 2 ]
print(" After unqiue value treatment :",len(num_list))

 
print(" Total number of count features before checking for SD=0 " ,len(count_list))
count_list=[column for column in count_list  if np.nanstd(X_train[column],axis=0) !=0.0 ]
print("Total number of count features after checking for SD=0 "  , len(count_list))

count_list=[column for column in count_list  if max(X_train[column])!=min(X_train[column])]
print("Total number of count  after checking for max=min "  , len(count_list))



[column for column in flag_list  if np.nanstd(X_train[column],axis=0) !=0.0 ]
flag_list=[column for column in flag_list  if max(X_train[column])!=min(X_train[column])]
print("Total number of flag features after checking for max=min "  , len(flag_list))

 

 Total number of numerical features before checking for SD=0  318
Total number of numerical features after checking for SD=0  318
Total number of numerical features after checking for max=min  318
 Before unqiue value treatment : 318
 After unqiue value treatment : 318
 Total number of count features before checking for SD=0  281
Total number of count features after checking for SD=0  281
Total number of count  after checking for max=min  281
Total number of flag features after checking for max=min  10


In [492]:
columns=num_list+count_list+flag_list
columns_dict={'columns': columns}
columns_pd  = pd.DataFrame(columns_dict,columns = (['columns']))
columns_pd.to_csv('D:/Prasanna/Prasanna/sepmodels/numlistsdmaxmin.csv', encoding='utf-8') 



In [499]:
columns=num_list
columns_dict={'columns': columns}
columns_pd  = pd.DataFrame(columns_dict,columns = (['columns']))
columns_pd.to_csv('D:/Prasanna/Prasanna/sepmodels/numlist_m1m2m3.csv', encoding='utf-8') 


columns=cat_list
columns_dict={'columns': columns}
columns_pd  = pd.DataFrame(columns_dict,columns = (['columns']))
columns_pd.to_csv('D:/Prasanna/Prasanna/sepmodels/catlistm1m2m3.csv', encoding='utf-8') 


In [462]:
#if % of zeros is greater than 90% omit:
print('num_list before',  len(num_list))
print('count_list before', len(count_list))
print('flag_list before' , len(flag_list))

num_list  =[column for column in num_list   if ((X_train[column] == 0).astype(int).sum(axis=0)/X_train.shape[0]) <=.90]
count_list=[column for column in count_list if ((X_train[column] == 0).astype(int).sum(axis=0)/X_train.shape[0]) <=.90]
flag_list =[column for column in flag_list  if ((X_train[column] == 0).astype(int).sum(axis=0)/X_train.shape[0]) <=.90]

print('num_list After',  len(num_list))
print('count_list After', len(count_list))
print('flag_list After' , len(flag_list))


num_list before 144
count_list before 281
flag_list before 14
num_list After 120
count_list After 247
flag_list After 6


In [390]:
columns=num_list
columns_dict={'columns': columns}
columns_pd  = pd.DataFrame(columns_dict,columns = (['columns']))
columns_pd.to_csv('D:/Prasanna/Prasanna/sepmodels/numlistzero.9.csv', encoding='utf-8') 


In [230]:
# Threshold for removing correlated variables
threshold = 0.9

# Absolute value correlation matrix
corr_matrix = X_train[num_list].corr().abs()
corr_matrix.head()

# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head()

# Select columns with correlations above threshold
corr_drop = [column for column in upper.columns if any(upper[column] > threshold)]
#
print('There are %d columns to remove.' % (len(corr_drop)))
#
#
 
num_list=[cols for cols in num_list if cols not in corr_drop]

There are 109 columns to remove.


In [ ]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
from __future__ import division 
#import statsmodels.api as sm
y_train_df=pd.DataFrame(y_train) 
#y_test_df.columns = ['dv'] 
VIF_train =  pd.concat([X_train[num_list].reset_index(drop=True),y_train_df], axis=1)
vif_list=list(VIF_train.columns.values)
print(len(vif_list))
xclude_list=['attrition_flag']
vif_list=[cols for cols in vif_list if cols not  in xclude_list]
#print(vif_list)
features="+".join(vif_list)

print(len(vif_list))

 

In [ ]:
qaz=VIF_train.columns.tolist() 
y, X = dmatrices('attrition_flag ~' + features, VIF_train, return_type="dataframe")
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
#print(vif)
vif_pd=pd.DataFrame(list(zip(vif,X.columns.values.tolist())))
vif_pd.columns=['vif_value','Feature']
xclude_list=['Intercept']

In [ ]:
var_list = vif_pd[vif_pd['vif_value'] <= 50.00]['Feature'].tolist()
var_list=[x for x in var_list if x not in xclude_list]

print(" Length of num variabls  after vif :" , len(var_list)) 
num_list=var_list

In [ ]:
log_list=X_train[num_list].columns[(X_train[num_list] < 0).any()].tolist()
#log transsformation
#for column in num_list:
#    X_train[column +'_log']=np.log(1 + X_train[column]).astype(float)
#    oot[column +'_log']    =np.log(1 + oot[column]).astype(float)
#    X_test[column +'_log'] =np.log(1 + X_test[column]).astype(float)

len(log_list)
for column in log_list:
    num_list.append(column+'_log')
    X_train[column +'_log']=np.log(1 + X_train[column]).astype(float)
    oot[column +'_log']    =np.log(1 + oot[column]).astype(float)
    X_test[column +'_log'] =np.log(1 + X_test[column]).astype(float)

In [463]:
X_train_scaled=X_train 
X_test_scaled=X_test
oot_scaled=oot

#scaler=MinMaxScaler()
#X_train_scaled[num_list]=scaler.fit_transform(X_train[num_list])
#X_test_scaled[num_list]=scaler.fit_transform(X_test[num_list]) 
#oot_scaled[num_list]=scaler.fit_transform(oot[num_list]) 


In [392]:
for col in num_list:
    percentiles = X_train_scaled[col].quantile([0.01,0.99]).values
    X_train_scaled[col][X_train_scaled[col] <= percentiles[0]] = percentiles[0]
    X_train_scaled[col][X_train_scaled[col] >= percentiles[1]] = percentiles[1]

    X_test_scaled[col][X_test_scaled[col] <= percentiles[0]] = percentiles[0]
    X_test_scaled[col][X_test_scaled[col] >= percentiles[1]] = percentiles[1]    
    
    oot_scaled[col][oot_scaled[col] <= percentiles[0]] = percentiles[0]
    oot_scaled[col][oot_scaled[col] >= percentiles[1]] = percentiles[1]

C:\Users\zcoe590\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\zcoe590\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5984: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\zcoe590\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2910: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pa

In [393]:
 

##Function to define fisher score calculation
def fishers_score(X_train,cols,dep_var):
    df=X_train
    x0=np.mean(df.loc[df[dep_var] == 0] [cols])
    var0=np.var(df.loc[df[dep_var] == 0][cols])
    x1=np.mean(df.loc[df[dep_var] == 1] [cols])
    var1=np.var(df.loc[df[dep_var] == 1][cols])
    try:
        score=abs(x0-x1)/(var0+var1)**(1/2)
    except ZeroDivisionError:
        score = 0
    return(score)

In [394]:
## For all numerical variables calculate fishers score
dv= 'attrition_flag'
fishers=[]
fishers_cutoff=.7
for column in num_list:
    #print(column)
    fishers.append(fishers_score(X_train,column,dv))    



In [395]:
fishers_dict =       {     'Variable'  : num_list,
                           'fisher'    : fishers
                      }
 
fishers_pd=pd.DataFrame(fishers_dict,columns = (['Variable','fisher']))

cutoff_rows=round((len(fishers)*fishers_cutoff)) + 1
fishers_pd = fishers_pd .sort_values(by='fisher', ascending=0)[1:cutoff_rows]

##########Check for correlations.Drop variable based on fisher scores
 
    
    
num_list=fishers_pd['Variable'].tolist()

In [464]:
print(len(num_list))

120


In [397]:
columns=num_list
columns_dict={'columns': columns}
columns_pd  = pd.DataFrame(columns_dict,columns = (['columns']))
columns_pd.to_csv('D:/Prasanna/Prasanna/sepmodels/numlistfischer.csv', encoding='utf-8') 


In [398]:
import math

Value=[]
Bins=[]	
Non_events=[]	
Events=[]	
percent_of_Non_Events=[]
percent_of_Events=[]
WOE=[]
IV=[]
cat_woe_var=[]
cat_woe_iv=[]



#qaz=cat_list[0:5]
 
for cat_var in cat_list:    
    X_train[cat_var+'_woe']=X_train[cat_var]
    #print("cat_var",cat_var)
    ## Create cross tab between the variable and dependent variable
    cat_pd=pd.crosstab(X_train[cat_var],X_train['attrition_flag'], margins=True)
    total_iv=0
    total_events=0
    total_non_events=0    
    woe_var=0
    iv_var=0
    num=0
    den=0    
    cat_to_woe={} 
    #(For all the rows from the cross tab except for the last row which is "ALL")
    for i in range(len(cat_pd.index)-1):        
        Value.append(cat_pd.index[i])
        Bins.append(i+1)
        Non_events.append(cat_pd[0][i])
        total_non_events+=cat_pd[0][i]
        Events.append(cat_pd[1][i])
        total_events+=cat_pd[1][i]
        percent_of_Non_Events.append(cat_pd[0][i]/cat_pd[0][len(cat_pd.index)-1])
        percent_of_Events.append(cat_pd[1][i]/cat_pd[1][len(cat_pd.index)-1])
        num=cat_pd[0][i]/cat_pd[0][len(cat_pd.index)-1]
        den=cat_pd[1][i]/cat_pd[1][len(cat_pd.index)-1]
        if num==0:
            woe_var=0
        elif den==0:
            woe_var= math.log(num/(den+.0001))
        else:
            woe_var=math.log(num/den)
        WOE.append(woe_var)
        iv_var=(cat_pd[0][i]/cat_pd[0][len(cat_pd.index)-1])-(cat_pd[1][i]/cat_pd[1][len(cat_pd.index)-1])
        IV.append(iv_var*woe_var)
        total_iv+=(iv_var*woe_var)
        #print("sri krisna")
        cat_to_woe[cat_pd.index[i]]=woe_var  
        X_train[cat_var+'_woe'] = X_train[cat_var].map(cat_to_woe)
       
        
    #cat_header.append("")
    Value.append(cat_var)
    Bins.append(cat_var)
    Non_events.append(total_non_events)
    Events.append(total_events)
    percent_of_Non_Events.append("***")
    percent_of_Events.append("***")
    WOE.append("*** ")
    IV.append(total_iv)
    cat_woe_var.append(cat_var)
    cat_woe_iv.append(total_iv)
    for gap in range(0,3):
        Value.append("   ")
        Bins.append("   ")
        Non_events.append("   ")
        Events.append("    ")
        percent_of_Non_Events.append("    ")
        percent_of_Events.append("    ")
        WOE.append("   ")
        IV.append("    ")
    Value.append("Value")
    Bins.append("Bins")
    Non_events.append("Non_events")
    Events.append("Events")
    percent_of_Non_Events.append("Percent_of_Non_Events")
    percent_of_Events.append("Percent_of_Events")
    WOE.append("Woe")
    IV.append("IV")    
        

C:\Users\zcoe590\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\zcoe590\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [399]:

cat_woe_dict = {      'Value'      : Value,
                      'Bins'      : Bins,
                      'Non_events': Non_events,
                      'Events'    : Events,
                      'Percent_of_Non_Events'      : percent_of_Non_Events,
                      'Percent_of_Events'         : percent_of_Events,
                      'Woe'       : WOE,
                      'IV'        : IV,  
                       }

###  2) Create a pandas dataframe using above dictionary

cat_woe  = pd.DataFrame(cat_woe_dict, columns = 
['Value','Bins','Non_events','Events','Percent_of_Non_Events','Percent_of_Events',
'Woe','IV'])        
      
           
    
cat_iv_dict={'Variable':cat_woe_var,'IV':cat_woe_iv}    
cat_iv=pd.DataFrame(cat_iv_dict,columns=['Variable','IV'])
cat_iv=cat_iv.sort_values(by='IV',ascending=0)
#cat_iv=cat_iv.query('IV>.1') 
#cat_iv=cat_iv.query('IV!=inf') 



In [400]:
cat_iv=cat_iv.query('IV>.01') 
cat_iv=cat_iv.query('IV!=inf') 
cat_list=cat_iv['Variable'].tolist()

In [401]:
columns=cat_list
columns_dict={'columns': columns}
columns_pd  = pd.DataFrame(columns_dict,columns = (['columns']))
columns_pd.to_csv('D:/Prasanna/Prasanna/sepmodels/catlist.01.csv', encoding='utf-8') 
 

In [465]:
features=num_list+age_list+cat_list+flag_list
features=num_list

In [466]:
#features=num_list+count_list+flag_list 
X_train_scaled=X_train_scaled[features]
X_test_scaled=X_test_scaled[features]
X_oot_scaled=oot_scaled[features]
Y_oot=oot['attrition_flag']
 
print(len(X_train_scaled.columns.tolist()))
print(len(X_test_scaled.columns.tolist()))
print(len(X_oot_scaled.columns.tolist()))

120
120
120


In [404]:
columns=features
columns_dict={'columns': columns}
columns_pd  = pd.DataFrame(columns_dict,columns = (['columns']))
columns_pd.to_csv('D:/Prasanna/Prasanna/sepmodels/featureslist.csv', encoding='utf-8') 


In [467]:
X_train_final = pd.get_dummies(X_train_scaled)
X_test_final  = pd.get_dummies(X_test_scaled)
X_oot_final   = pd.get_dummies(X_oot_scaled)

#X_train_final = X_train_scaled
#X_test_final  = X_test_scaled
#X_oot_final   = X_oot_scaled

In [ ]:
train_features = X_train_final[var]
test_features = X_test_final[var]
oot_features=X_oot_final[var]


features = X_train_final[var].columns.values 
for feature in features:
    mean, std = train_features[feature].mean(), train_features[feature].std()
    train_features.loc[:, feature] = (train_features[feature] - mean) / std
    test_features.loc[:, feature] = (test_features[feature] - mean) / std 
    oot_features.loc[:, feature] = (oot_features[feature] - mean) / std 

train_features=train_features.as_matrix()                      
test_features=test_features.as_matrix() 
oot_features=oot_features.as_matrix()  


In [ ]:
train_labels=pd.DataFrame()
train_labels['attrition_flag']=y_train
test_labels=pd.DataFrame()
test_labels['attrition_flag']=y_test
oot_labels=pd.DataFrame()
oot_labels['attrition_flag']=oot['attrition_flag_


In [ ]:
def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)
    # TODO: Implement batching
    output_batches = []
    
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        output_batches.append(batch)
        
    #return output_batches
    return output_batches

In [ ]:
n_features=len(X_train_final[var].columns.tolist())
n_labels=1
 
beta=.01

''' Three layer model
r = (30/2)**(1/4) = 1.9679896712654303 = 2
HL1 = 2*2**3 = 16
HL2 = 2*2**2 = 8
HL3 = 2*2**1 = 4
'''

n_hidden_nodes1 =27
n_hidden_nodes2 = 9
n_hidden_nodes3 = 3

features = tf.placeholder(tf.float32,[None,n_features])
labels = tf.placeholder(tf.float32,[None,n_labels])


In [ ]:
n_features

In [ ]:
 
# layer 1
keep_prob = tf.placeholder(tf.float32) 

hidden_layer1_weights=tf.Variable(tf.truncated_normal([n_features,n_hidden_nodes1],stddev = 0.15))
hidden_layer1_bias=tf.Variable(tf.truncated_normal([n_hidden_nodes1]))
#hidden_layer1_weights=tf.Variable(tf.truncated_normal([n_features,n_labels],stddev = 0.15))
#hidden_layer1_bias=tf.Variable(tf.truncated_normal([n_labels]))
hidden_layer1=tf.nn.sigmoid(tf.add(tf.matmul(features,hidden_layer1_weights),hidden_layer1_bias))
hidden_layer1 = tf.nn.dropout(hidden_layer1, keep_prob)

print(hidden_layer1_weights.shape)
print(hidden_layer1_bias.shape)

In [ ]:

# layer 2

hidden_layer2_weights=tf.Variable(tf.truncated_normal([n_hidden_nodes1,n_hidden_nodes2],stddev = 0.15))
hidden_layer2_bias=tf.Variable(tf.truncated_normal([n_hidden_nodes2]))
#hidden_layer2_weights=tf.Variable(tf.truncated_normal([n_hidden_nodes1,n_labels],stddev = 0.15))
#hidden_layer2_bias=tf.Variable(tf.truncated_normal([n_labels]))
hidden_layer2=tf.nn.sigmoid(tf.matmul(hidden_layer1,hidden_layer2_weights)+hidden_layer2_bias)
hidden_layer2 = tf.nn.dropout(hidden_layer2, keep_prob)

print(hidden_layer2_weights.shape)
print(hidden_layer2_bias.shape)

In [ ]:

# layer 3

#hidden_layer3_weights=tf.Variable(tf.truncated_normal([n_hidden_nodes2,n_labels],stddev = 0.15))
#hidden_layer3_bias=tf.Variable(tf.truncated_normal([n_labels]))
hidden_layer3_weights=tf.Variable(tf.truncated_normal([n_hidden_nodes2,n_hidden_nodes3],stddev = 0.15))
hidden_layer3_bias=tf.Variable(tf.truncated_normal([n_hidden_nodes3]))
hidden_layer3=tf.nn.sigmoid(tf.add(tf.matmul(hidden_layer2,hidden_layer3_weights),hidden_layer3_bias))
hidden_layer3 = tf.nn.dropout(hidden_layer3, keep_prob)

print(hidden_layer3_weights.shape)
print(hidden_layer3_bias.shape)


In [ ]:
# layer 4

hidden_layer4_weights=tf.Variable(tf.truncated_normal([n_hidden_nodes3,n_labels],stddev = 0.15))
hidden_layer4_bias=tf.Variable(tf.truncated_normal([n_labels]))
hidden_layer4=tf.nn.sigmoid(tf.matmul(hidden_layer3,hidden_layer4_weights)+hidden_layer4_bias)
hidden_layer4 = tf.nn.dropout(hidden_layer4, keep_prob)

print(hidden_layer4_weights.shape)
print(hidden_layer4_bias.shape)


In [ ]:
learning_rate=.000001   
#cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hidden_layer4, labels=labels))
cost= tf.nn.sigmoid_cross_entropy_with_logits(logits=hidden_layer4, labels=labels)
#regularizers = (tf.nn.l2_loss(hidden_layer1_weights) + tf.nn.l2_loss(hidden_layer2_weights))/2 
#cost = tf.reduce_mean(cost + beta * regularizers)
cost = tf.reduce_mean(cost)

#optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
optimizer =  tf.train.AdamOptimizer(learning_rate).minimize(cost)



In [ ]:
batch_size = 2810*12

sess=tf.Session()


init=tf.global_variables_initializer()
init1=tf.initialize_local_variables()
sess.run(init)
sess.run(init1) 

 

for epoch_i in range(35000):
    for batch_features, batch_labels in batches(batch_size, train_features, train_labels):
        sess.run([optimizer], feed_dict={features: batch_features, labels: batch_labels,keep_prob:.7})
   
    if epoch_i % 100 == 0:
        print("epoch",epoch_i)
        current_cost = sess.run(cost,feed_dict={features: batch_features, labels: batch_labels,keep_prob:1})
        print("Training Current Cost:" ,current_cost)
         
        pred = sess.run(hidden_layer4,feed_dict={features: train_features, labels: train_labels,keep_prob:1})
        pred=pd.DataFrame(list(pred))[0]
        print("AUC Score (Train): %f" % roc_auc_score(train_labels['attrition_flag'], pred))
        
        pred = sess.run(hidden_layer4,feed_dict={features: oot_features, labels: oot_labels,keep_prob:1})
        pred=pd.DataFrame(list(pred))[0]
        print("AUC Score (Test): %f" % roc_auc_score(oot_labels['attrition_flag'], pred))
        current_cost = sess.run(cost,feed_dict={features: oot_features, labels: oot_labels,keep_prob:1})
        print("Test Cost:" ,current_cost)
        
pred=pd.DataFrame(list(pred))[0]
print("AUC Score (Test): %f" % roc_auc_score(test_labels['attrition_flag'], pred))
